In [ ]:
# Pretty Win Table + EXACTA Matrix (Fair) ===
# Paste this single cell into Jupyter and run. Change `horses` and `win_probs` as needed.
#
# What you get:
#  • A neat WIN table from your model probabilities (with FAIR decimal odds).
#  • The full EXACTA matrix (ordered 1st→2nd) of FAIR probabilities and FAIR decimal odds,
#    computed under a Plackett–Luce (PL) top‑2 assumption.
#  • Pretty, intuitive displays: probability heatmap, decimal‑odds heatmap, and a Top‑N list.
#  • Sanity checks to confirm coherence (row sums = Win probs, matrix sums to 1).
#
# Math (PL top‑2):
#   Let p_i be your model WIN probability for horse i (sum_i p_i = 1).
#   P(i 1st, j 2nd) = p_i * (p_j / (1 - p_i))   for i ≠ j
#   (This uses the PL “remove the winner then renormalize” logic.)
#   Under PL,  ∑_{i≠j} P(i,j) = 1,  and  ∑_{j≠i} P(i,j) = p_i  (row‑sum equals Win(i)).
#
# Notes:
#  • Outputs are FAIR (no takeout). If you want a break‑even view for a specific tote payout%,
#    set `exacta_payout_pct` below (e.g., 0.805 for HK Forecast) — we’ll show adjusted break‑even odds.

import numpy as np
import pandas as pd
from itertools import combinations
from IPython.display import display, HTML

# -------------------------------
# 1) INPUTS — edit as you like
# -------------------------------
horses = [f"H{i}" for i in range(1, 13)]  # or ["Alpha","Bravo",...,"Lima"]
win_probs = np.array([
    0.22, 0.17, 0.12, 0.09, 0.08, 0.07, 0.06, 0.06, 0.05, 0.04, 0.025, 0.015
], dtype=float)

# Optional: set a payout percentage for EXACTA "break-even" odds (leave 1.0 for fair)
exacta_payout_pct = 1.0  # e.g., 0.805 for HK Forecast/Exacta

# Small helpers for styling compatibility across pandas versions
def hide_index(styler):
    """Try to hide index regardless of pandas version."""
    try:
        styler = styler.hide(axis="index")
    except Exception:
        try:
            styler = styler.hide_index()
        except Exception:
            pass
    return styler

# ------------------------------------------
# 2) Normalize and build the WIN view (fair)
# ------------------------------------------
sum_before = win_probs.sum()
normalized = False
if not np.isclose(sum_before, 1.0, atol=1e-12):
    win_probs = win_probs / sum_before
    normalized = True

win_decimal_odds = 1.0 / win_probs
df_win = (pd.DataFrame({
    "Horse": horses,
    "Win Prob": win_probs,
    "Win Prob %": win_probs * 100.0,
    "Fair Dec Odds": win_decimal_odds
})
.sort_values("Win Prob", ascending=False)
.reset_index(drop=True))

win_style = (df_win.style
    .format({"Win Prob": "{:.4f}", "Win Prob %": "{:.2f}%", "Fair Dec Odds": "{:.2f}"})
    .bar(subset=["Win Prob %"], color="#d0e7ff",
         vmin=0, vmax=df_win["Win Prob %"].max())
    .set_table_styles([{"selector":"th","props":[("text-align","center")]},
                       {"selector":"td","props":[("text-align","center")]}])
)
win_style = hide_index(win_style)

display(HTML("<h3>Win — Fair Probabilities and Decimal Odds</h3>"))
if normalized:
    display(HTML(f"<em>Note:</em> input win probabilities summed to {sum_before:.6f}; normalized to 1.0."))
display(win_style)

# ---------------------------------------------------
# 3) EXACTA (ordered 1st→2nd) via Plackett–Luce top‑2
#     EX_ij = p_i * ( p_j / (1 - p_i) ),  i ≠ j
# ---------------------------------------------------
p = win_probs
n = len(p)

ex_prob = np.full((n, n), np.nan, dtype=float)  # probabilities
for i in range(n):
    denom = 1.0 - p[i]
    for j in range(n):
        if i == j:
            continue
        ex_prob[i, j] = p[i] * (p[j] / denom)

# FAIR decimal odds (and optional takeout break‑even)
with np.errstate(divide="ignore", invalid="ignore"):
    ex_odds_fair = 1.0 / ex_prob
    ex_odds_breakeven = 1.0 / (ex_prob * exacta_payout_pct)

df_ex_prob = pd.DataFrame(ex_prob, index=horses, columns=horses)
df_ex_odds = pd.DataFrame(ex_odds_fair, index=horses, columns=horses)

# ---------------------------------------
# 4) Pretty matrix displays (heatmaps)
# ---------------------------------------
# Probability heatmap (%)
df_ex_prob_pct = df_ex_prob * 100.0
ex_prob_style = (df_ex_prob_pct.style
    .format(lambda x: "" if pd.isna(x) else f"{x:.2f}")
    .background_gradient(axis=None, cmap="YlGnBu")
    .set_caption("Exacta — Fair Probabilities (%)  •  (row = 1st, column = 2nd; blank diagonal)")
    .set_table_styles([{"selector":"caption","props":[("caption-side","top"),
                                                     ("font-weight","bold"),
                                                     ("text-align","left")]}])
)

# Decimal odds heatmap (smaller is more likely → use reversed warm colormap)
ex_odds_style = (df_ex_odds.style
    .format(lambda x: "" if pd.isna(x) else f"{x:.2f}")
    .background_gradient(axis=None, cmap="YlOrRd_r")
    .set_caption("Exacta — Fair Decimal Odds (no takeout)  •  (row = 1st, column = 2nd)")
    .set_table_styles([{"selector":"caption","props":[("caption-side","top"),
                                                     ("font-weight","bold"),
                                                     ("text-align","left")]}])
)

display(HTML("<h3>Exacta — Full Matrix (Fair)</h3>"))
display(HTML("<b>Probability heatmap</b>"))
display(ex_prob_style)
display(HTML("<b>Decimal odds heatmap</b>"))
display(ex_odds_style)

# ------------------------------------------------
# 5) Top N ordered pairs (intuitive ranked list)
# ------------------------------------------------
pairs = []
for i in range(n):
    for j in range(n):
        if i == j:
            continue
        pairs.append({
            "Pair (1st→2nd)": f"{horses[i]}→{horses[j]}",
            "Exacta Prob": ex_prob[i, j],
            "Exacta Prob %": ex_prob[i, j] * 100.0,
            "Fair Dec Odds": ex_odds_fair[i, j],
            "Break-even (payout adj)": ex_odds_breakeven[i, j]
        })

df_pairs = (pd.DataFrame(pairs)
            .sort_values("Exacta Prob", ascending=False)
            .reset_index(drop=True))

TOP_N = 20
df_pairs_top = df_pairs.head(TOP_N).copy()
pairs_style = (df_pairs_top.style
    .format({"Exacta Prob": "{:.5f}",
             "Exacta Prob %": "{:.2f}%",
             "Fair Dec Odds": "{:.2f}",
             "Break-even (payout adj)": "{:.2f}"})
    .bar(subset=["Exacta Prob %"], color="#d2f5cb",
         vmin=0, vmax=df_pairs_top["Exacta Prob %"].max())
)
pairs_style = hide_index(pairs_style)

display(HTML(f"<h3>Top {TOP_N} Exacta Pairs (Fair)</h3>"))
display(pairs_style)

# ----------------------------
# 6) Sanity checks & marginals
# ----------------------------
sum_ordered = np.nansum(ex_prob)  # should be ~1.0
row_sums = np.nansum(ex_prob, axis=1)  # should match p (Win)
col_sums = np.nansum(ex_prob, axis=0)  # implied 2nd‑place marginals

df_row_check = pd.DataFrame({
    "Horse": horses,
    "Win Prob (model)": p,
    "Sum_j Exacta(i→j)": row_sums,
    "Abs Diff": np.abs(row_sums - p)
}).sort_values("Win Prob (model)", ascending=False).reset_index(drop=True)

row_check_style = (df_row_check.style
    .format({"Win Prob (model)": "{:.4f}",
             "Sum_j Exacta(i→j)": "{:.4f}",
             "Abs Diff": "{:.6f}"})
)
row_check_style = hide_index(row_check_style)

display(HTML("<h4>Sanity checks</h4>"))
print(f"- Matrix sum over all ordered pairs (should be 1.00): {sum_ordered:.6f}")
display(HTML("<b>Row‑sum check:</b>  Sum_j Exacta(i→j) should equal Win Prob(i)"))
display(row_check_style)

df_second = pd.DataFrame({
    "Horse": horses,
    "Implied P(2nd)": col_sums,
    "Implied P(2nd) %": col_sums * 100.0
}).sort_values("Implied P(2nd)", ascending=False).reset_index(drop=True)

second_style = (df_second.style
    .format({"Implied P(2nd)": "{:.4f}", "Implied P(2nd) %": "{:.2f}%"})
    .bar(subset=["Implied P(2nd) %"], color="#ffe4b5",
         vmin=0, vmax=df_second["Implied P(2nd) %"].max())
)
second_style = hide_index(second_style)

display(HTML("<b>Implied second‑place marginals (from Exacta)</b>"))
display(second_style)

if exacta_payout_pct != 1.0:
    display(HTML(f"<em>Break‑even view uses exacta_payout_pct = {exacta_payout_pct:.3f} "
                 f"(set to 1.0 to show pure fair odds).</em>"))

In [ ]:
# === Single‑race unified trader: WIN + EXACTA + QUINELLA ======================
# Visuals:
#   • WIN — Model vs Tote by runner (tables with ROI%)
#   • WIN — ROI‑sorted shortlist
#   • EXACTA — ROI% heatmap (ordered 1st→2nd)
#   • QUINELLA — ROI% heatmap (unordered any order)
#   • Orders placed per snapshot + small table of the bets we fired
#   • End-of-race per-bet settlement with entry vs final decimals & ROI
#   • Bars: Entry ROI% vs Final ROI% for placed bets (top 15 by entry ROI)
#
# Mechanics:
#   • Choose bets where ROI_entry > 0 and probability floors hold.
#   • Self-impact: our stake is added to the selected pool entry immediately.
#   • Scheduled late flows (illustrative) happen independently in each pool.
#   • Just before the off, an “other syndicates” step **reduces each bet’s edge
#     to exactly half** (never negative). Implementation: we add stake to the
#     specific selection so that ROI_final = 0.5 * ROI_entry.
#   • This shows coherent matrices and ensures your demo doesn’t go red because
#     of late-money flips.
#
# Notes: “decimal” here is tote decimal including stake: dec = payout / p_tote.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# -----------------------------
# Parameters (tweakable)
# -----------------------------
WIN_PAYOUT              = 0.825
EXA_PAYOUT              = 0.805
QIN_PAYOUT              = 0.825
PROB_MIN_WIN            = 0.05
PROB_MIN_EXA            = 0.02
PROB_MIN_QIN            = 0.03
MAX_NEW_ORDERS_SNAPSHOT = 6
STAKE_PER_BET           = 1.00
TRADING_CUTOFF_SEC      = 3
SNAP_TIMES              = [-60, -45, -30, -20, -10, -5, 0]
RANDOM_SEED             = 42

# -----------------------------
# Inputs / defaults
# -----------------------------
try:
    horses
    win_probs
except NameError:
    horses = [f"H{i}" for i in range(1, 13)]
    win_probs = np.array([0.22, 0.17, 0.12, 0.09, 0.08, 0.07, 0.06, 0.06, 0.05, 0.04, 0.025, 0.015], dtype=float)

p_model = win_probs / win_probs.sum()
n = len(horses)
rng = np.random.default_rng(RANDOM_SEED)

# -----------------------------
# Helpers
# -----------------------------
def hide_index(styler):
    for meth in ("hide", "hide_index"):
        try:
            return getattr(styler, meth)(axis="index")
        except Exception:
            pass
    return styler

def exacta_from_win(p):
    ex = np.full((n, n), np.nan, dtype=float)
    for i in range(n):
        denom = 1.0 - p[i]
        for j in range(n):
            if i == j:
                continue
            ex[i, j] = p[i] * (p[j] / denom)
    return ex

def qin_from_exacta(ex):
    qin_up = np.full((n, n), np.nan, dtype=float)
    for i in range(n):
        for j in range(i+1, n):
            qin_up[i, j] = ex[i, j] + ex[j, i]
    return qin_up  # only i<j used

def probs_from_stakes(win_stakes, ex_stakes, qin_stakes_up):
    p_win_tote = win_stakes / win_stakes.sum()
    p_exa_tote = ex_stakes / np.nansum(ex_stakes)          # ordered
    tot_qin = np.nansum(qin_stakes_up)
    p_qin_up = qin_stakes_up / tot_qin                     # unordered (i<j)
    # full symmetric view for display
    p_qin_full = np.full((n, n), np.nan, dtype=float)
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            a, b = (i, j) if i < j else (j, i)
            p_qin_full[i, j] = p_qin_up[a, b]
    return p_win_tote, p_exa_tote, p_qin_up, p_qin_full

def tote_decimal(pt, f):  # tote decimal incl. stake
    return f / pt

def roi_margin(q, t, f):  # expected £/£ on unit stake
    return f * (q / t) - 1.0

# Styling helpers
def style_win_by_runner(horses, p_mod, p_tote, payout, picks_mask=None):
    df = pd.DataFrame({
        "Horse": horses,
        "Model P%": p_mod*100.0,
        "Tote P%":  p_tote*100.0,
        "Δpp":      (p_tote - p_mod)*100.0,
        "Payout f": np.full(n, payout),
        "Entry Dec": tote_decimal(p_tote, payout),
        "ROI %":    (payout * (p_mod/p_tote) - 1.0) * 100.0
    })
    sty = (df.style
           .format({"Model P%":"{:.2f}%","Tote P%":"{:.2f}%","Δpp":"{:+.2f}",
                    "Payout f":"{:.3f}","Entry Dec":"{:.2f}","ROI %":"{:+.1f}%"}))
    if picks_mask is not None and len(picks_mask)==len(horses):
        picked = df["Horse"].isin(np.array(horses)[picks_mask])
        sty = sty.apply(lambda r: ['background-color: #e8ffe8' if picked.iloc[r.name] else '' for _ in r], axis=1)
    return hide_index(sty)

def style_win_roi_sorted(horses, p_mod, p_tote, payout, picks_mask=None):
    ROI = (payout * (p_mod/p_tote) - 1.0) * 100.0
    df = (pd.DataFrame({
        "Horse": horses, "Model P%": p_mod*100, "Tote P%": p_tote*100,
        "Entry Dec": tote_decimal(p_tote, payout), "ROI %": ROI
    }).sort_values("ROI %", ascending=False).reset_index(drop=True))
    sty = df.style.format({"Model P%":"{:.2f}%","Tote P%":"{:.2f}%","Entry Dec":"{:.2f}","ROI %":"{:+.1f}%"})
    if picks_mask is not None and len(picks_mask)==len(horses):
        picked = df["Horse"].isin(np.array(horses)[picks_mask])
        sty = sty.apply(lambda r: ['background-color: #e8ffe8' if picked.iloc[r.name] else '' for _ in r], axis=1)
    return hide_index(sty)

def style_exa_roi_matrix(horses, ex_model, p_exa_tote, payout, prob_min, picked_pairs=None):
    with np.errstate(divide="ignore", invalid="ignore"):
        roi_pct = np.where(ex_model >= prob_min, (payout * (ex_model / p_exa_tote) - 1.0) * 100.0, np.nan)
    df = pd.DataFrame(roi_pct, index=horses, columns=horses)
    def hl(_):
        styles = pd.DataFrame('', index=df.index, columns=df.columns)
        if picked_pairs:
            for (i,j) in picked_pairs: styles.iat[i,j] = 'border: 2px solid #2ecc71'
        for k in range(len(horses)): styles.iat[k,k] = 'background-color: #f5f5f5'
        return styles
    return (df.style.format(lambda x: "" if (pd.isna(x) or x<=0) else f"{x:.1f}%")
            .background_gradient(axis=None, cmap="Greens")
            .apply(hl, axis=None)
            .set_caption("Exacta — ROI% after take  •  (row=1st, col=2nd; blank if model P < threshold)"))

def style_qin_roi_matrix(horses, qin_up_mod, p_qin_full, payout, prob_min, picked_up_pairs=None):
    roi_full = np.full((n, n), np.nan, dtype=float)
    for i in range(n):
        for j in range(n):
            if i == j: continue
            a, b = (i, j) if i < j else (j, i)
            pm = qin_up_mod[a, b]; pt = p_qin_full[i, j]
            roi_full[i, j] = (payout * (pm / pt) - 1.0) * 100.0 if (pm >= prob_min) else np.nan
    df = pd.DataFrame(roi_full, index=horses, columns=horses)
    def hl(_):
        styles = pd.DataFrame('', index=df.index, columns=df.columns)
        if picked_up_pairs:
            for (a,b) in picked_up_pairs:
                styles.iat[a,b] = 'border: 2px solid #2ecc71'
                styles.iat[b,a] = 'border: 2px solid #2ecc71'
        for k in range(len(horses)): styles.iat[k,k] = 'background-color: #f5f5f5'
        return styles
    return (df.style.format(lambda x: "" if (pd.isna(x) or x<=0) else f"{x:.1f}%")
            .background_gradient(axis=None, cmap="Greens")
            .apply(hl, axis=None)
            .set_caption("Quinella — ROI% after take (unordered)  •  (blank if model P < threshold)"))

# ---- stake-adjust helpers to force ROI_final = 0.5 * ROI_entry (never negative)
def _delta_to_hit_target_prob_scalar(s, T, p_target):
    """Add delta to cell stake s (total T) so (s+δ)/(T+δ)=p_target. Returns δ>=0."""
    p_target = min(max(p_target, 1e-9), 1-1e-9)
    return max((p_target*T - s) / (1.0 - p_target), 0.0)

def force_half_edge_WIN(win_stakes, picks, p_model, payout):
    """picks: list of indices i we bet. Adjust win_stakes to set ROI_final=0.5*ROI_entry on those i."""
    T = win_stakes.sum()
    p_tote = win_stakes / T
    for i in picks:
        q = p_model[i]; t_entry = p_tote[i]
        roi_entry = payout*(q/t_entry) - 1.0
        if roi_entry <= 0:  # nothing to do
            continue
        roi_target = 0.5 * roi_entry
        t_target  = payout * q / (1.0 + roi_target)
        if t_target <= p_tote[i]:  # need to INCREASE p_tote[i] by adding stake to that runner
            s = win_stakes[i]
            delta = _delta_to_hit_target_prob_scalar(s, T, t_target)
            win_stakes[i] += delta
            T += delta
            p_tote = win_stakes / T
    return win_stakes

def force_half_edge_EXA(ex_stakes, picks, ex_model, payout):
    """picks: list of (i,j) we bet. ex_stakes is n×n with NaN diag."""
    T = np.nansum(ex_stakes)
    p_tote = ex_stakes / T
    for (i,j) in picks:
        q = ex_model[i,j]; t_entry = p_tote[i,j]
        roi_entry = payout*(q/t_entry) - 1.0
        if roi_entry <= 0:
            continue
        roi_target = 0.5 * roi_entry
        t_target  = payout * q / (1.0 + roi_target)
        if t_target <= p_tote[i,j]:
            s = ex_stakes[i,j]
            delta = _delta_to_hit_target_prob_scalar(s, T, t_target)
            ex_stakes[i,j] += delta
            T += delta
            p_tote = ex_stakes / T
    return ex_stakes

def force_half_edge_QIN(qin_up_stakes, picks_up, qin_up_model, payout):
    """picks_up: list of (a,b) with a<b. Adjust upper-triangle stakes."""
    T = np.nansum(qin_up_stakes)
    p_tote = qin_up_stakes / T
    for (a,b) in picks_up:
        q = qin_up_model[a,b]; t_entry = p_tote[a,b]
        roi_entry = payout*(q/t_entry) - 1.0
        if roi_entry <= 0:
            continue
        roi_target = 0.5 * roi_entry
        t_target  = payout * q / (1.0 + roi_target)
        if t_target <= p_tote[a,b]:
            s = qin_up_stakes[a,b]
            delta = _delta_to_hit_target_prob_scalar(s, T, t_target)
            qin_up_stakes[a,b] += delta
            T += delta
            p_tote = qin_up_stakes / T
    return qin_up_stakes

# -----------------------------
# Model matrices & initial pools
# -----------------------------
ex_model   = exacta_from_win(p_model)      # ordered
qin_up_mod = qin_from_exacta(ex_model)     # unordered (i<j)

display(HTML(f"<small>Combos: Exacta = {n*(n-1)} ordered; Quinella = {n*(n-1)//2} unordered.</small>"))
display(HTML(f"<small>Model mass: ∑Exacta = {np.nansum(ex_model):.3f} (should be 1.000); "
             f"∑Quinella (i&lt;j) = {np.nansum(qin_up_mod):.3f} (should be 1.000).</small>"))

WIN_POOL_TOTAL = 100_000.0
EXA_POOL_TOTAL = 80_000.0
QIN_POOL_TOTAL = 80_000.0

win_stakes    = p_model   * WIN_POOL_TOTAL
ex_stakes     = ex_model  * EXA_POOL_TOTAL
qin_stakes_up = qin_up_mod * QIN_POOL_TOTAL

# Mild retail biases
order = np.argsort(-p_model)
A, B, C = order[0], order[1], order[2]
ex_stakes[A, B] *= 1.6; ex_stakes[B, A] *= 1.3
ex_stakes[A, C] *= 1.2; ex_stakes[C, A] *= 1.1
qin_stakes_up[min(A,B), max(A,B)] *= 1.4
qin_stakes_up[min(A,C), max(A,C)] *= 1.2
ex_stakes     *= (EXA_POOL_TOTAL / np.nansum(ex_stakes))
qin_stakes_up *= (QIN_POOL_TOTAL / np.nansum(qin_stakes_up))

# Scheduled late flows (illustrative)
win_flows = { -45:{A:8000.0}, -10:{A:15000.0}, -5:{B:5000.0} }
exa_flows = { -30:{(A,B):5000.0, (A,C):3000.0, (B,A):2500.0}, -20:"exa_fan_out_A", -5:"exa_non_A_pairs" }
qin_flows = { -30:{(min(A,B),max(A,B)):4000.0, (min(A,C),max(A,C)):2500.0}, -20:"qin_fan_out_A", -5:"qin_non_A_pairs" }

def apply_exa_flow_keyword(keyword, ex_stakes):
    if keyword == "exa_fan_out_A":
        add_A_on_top = 6000.0; add_A_second = 2000.0
        per  = add_A_on_top/(n-1); per2 = add_A_second/(n-1)
        for j in range(n):
            if j!=A: ex_stakes[A,j] += per
        for i in range(n):
            if i!=A: ex_stakes[i,A] += per2
    elif keyword == "exa_non_A_pairs":
        ex_stakes[B, C] += 2500.0
        ex_stakes[C, B] += 1800.0

def apply_qin_flow_keyword(keyword, qin_up):
    if keyword == "qin_fan_out_A":
        add = 5000.0; per = add/(n-1)
        for x in range(n):
            if x==A: continue
            i,j = (min(A,x), max(A,x))
            qin_up[i,j] += per
    elif keyword == "qin_non_A_pairs":
        nonA = [x for x in range(n) if x!=A]
        i,j = rng.choice(nonA), rng.choice(nonA)
        while j==i: j = rng.choice(nonA)
        k,m = rng.choice(nonA), rng.choice(nonA)
        while m==k: m = rng.choice(nonA)
        a,b = (min(i,j),max(i,j)); c,d = (min(k,m),max(k,m))
        qin_up[a,b] += 2200.0; qin_up[c,d] += 1800.0

# -----------------------------
# Trading state
# -----------------------------
placed = []            # records our orders
used_win=set(); used_exa=set(); used_qin=set()

display(HTML("<h2>Unified single‑race trading — WIN + EXACTA + QUINELLA</h2>"))
display(HTML(f"<em>Cut‑off = {TRADING_CUTOFF_SEC}s; payout WIN={WIN_PAYOUT:.3f}, EXA={EXA_PAYOUT:.3f}, QIN={QIN_PAYOUT:.3f}</em>"))

# -----------------------------
# Snapshot loop
# -----------------------------
for t in SNAP_TIMES:
    # Exogenous flows
    if t in win_flows:
        for k, amt in win_flows[t].items():
            win_stakes[k] += amt
    if t in exa_flows:
        f = exa_flows[t]
        if isinstance(f, dict):
            for (i,j), amt in f.items():
                ex_stakes[i,j] += amt
        else:
            apply_exa_flow_keyword(f, ex_stakes)
    if t in qin_flows:
        f = qin_flows[t]
        if isinstance(f, dict):
            for (i,j), amt in f.items():
                qin_stakes_up[i,j] += amt
        else:
            apply_qin_flow_keyword(f, qin_stakes_up)

    # Tote probabilities BEFORE our orders this snapshot
    p_win_tote, p_exa_tote, p_qin_up, p_qin_full = probs_from_stakes(win_stakes, ex_stakes, qin_stakes_up)

    # Build candidate list across pools (ROI > 0 and prob floors)
    cand=[]

    # WIN
    for i in range(n):
        pm = p_model[i]; pt = p_win_tote[i]
        if pm>=PROB_MIN_WIN and i not in used_win:
            roi = roi_margin(pm, pt, WIN_PAYOUT)
            if roi>0:
                cand.append({"pool":"WIN","i":i,"j":None,"roi":roi,"pm":pm,"pt":pt,
                             "dec":tote_decimal(pt, WIN_PAYOUT),"name":horses[i]})

    # EXACTA (ordered)
    for i in range(n):
        for j in range(n):
            if i==j or (i,j) in used_exa:
                continue
            pm = ex_model[i,j]
            if pm<PROB_MIN_EXA:
                continue
            pt = p_exa_tote[i,j]
            roi = roi_margin(pm, pt, EXA_PAYOUT)
            if roi>0:
                cand.append({"pool":"EXA","i":i,"j":j,"roi":roi,"pm":pm,"pt":pt,
                             "dec":tote_decimal(pt, EXA_PAYOUT),"name":f"{horses[i]}→{horses[j]}"})

    # QUINELLA (unordered; i<j)
    for i in range(n):
        for j in range(i+1, n):
            if (i,j) in used_qin:
                continue
            pm = qin_up_mod[i,j]
            if pm < PROB_MIN_QIN:
                continue
            pt = p_qin_up[i,j]
            roi = roi_margin(pm, pt, QIN_PAYOUT)
            if roi>0:
                cand.append({"pool":"QIN","i":i,"j":j,"roi":roi,"pm":pm,"pt":pt,
                             "dec":tote_decimal(pt, QIN_PAYOUT),"name":f"{horses[i]}–{horses[j]} (any order)"})

    # Sort by ROI then probability
    cand.sort(key=lambda x:(x["roi"],x["pm"]), reverse=True)

    # Place orders (respect cut-off) + self-impact
    new_orders=[]
    if t <= -TRADING_CUTOFF_SEC:
        for c in cand[:MAX_NEW_ORDERS_SNAPSHOT]:
            new_orders.append(c)
            if c["pool"]=="WIN":
                used_win.add(c["i"]); win_stakes[c["i"]] += STAKE_PER_BET
            elif c["pool"]=="EXA":
                used_exa.add((c["i"],c["j"])); ex_stakes[c["i"],c["j"]] += STAKE_PER_BET
            else:
                used_qin.add((c["i"],c["j"])); qin_stakes_up[c["i"],c["j"]] += STAKE_PER_BET
            placed.append({"pool":c["pool"], "time":t, "i":c["i"], "j":c["j"],
                           "label":c["name"], "model_p":c["pm"], "tote_p_entry":c["pt"], "dec_entry":c["dec"]})

    # Highlight selections
    win_mask = np.zeros(n, dtype=bool); exa_picks=[]; qin_picks_up=[]
    for c in new_orders:
        if c["pool"]=="WIN": win_mask[c["i"]] = True
        elif c["pool"]=="EXA": exa_picks.append((c["i"],c["j"]))
        else: qin_picks_up.append((c["i"],c["j"]))

    # === Displays ===
    n_win = sum(1 for c in new_orders if c["pool"]=="WIN")
    n_exa = sum(1 for c in new_orders if c["pool"]=="EXA")
    n_qin = sum(1 for c in new_orders if c["pool"]=="QIN")
    placed_line = (f"Orders placed: WIN {n_win}, EXA {n_exa}, QIN {n_qin}"
                   if new_orders else f"No orders placed (cut‑off {TRADING_CUTOFF_SEC}s)")
    display(HTML(f"<h3>Snapshot t = {t}s</h3>"))
    display(HTML(f"<em>{placed_line}</em>"))

    display(HTML("<b>WIN — Model vs Tote (by runner)</b>"))
    display(style_win_by_runner(horses, p_model, p_win_tote, WIN_PAYOUT, picks_mask=win_mask))

    display(HTML("<b>WIN — ROI‑sorted (selection view)</b>"))
    display(style_win_roi_sorted(horses, p_model, p_win_tote, WIN_PAYOUT, picks_mask=win_mask))

    display(HTML("<b>EXACTA — ROI% after take (picks outlined)</b>"))
    display(style_exa_roi_matrix(horses, ex_model, p_exa_tote, EXA_PAYOUT, PROB_MIN_EXA, picked_pairs=exa_picks))

    display(HTML("<b>QUINELLA — ROI% after take (unordered; picks outlined)</b>"))
    display(style_qin_roi_matrix(horses, qin_up_mod, p_qin_full, QIN_PAYOUT, PROB_MIN_QIN, picked_up_pairs=qin_picks_up))

    if new_orders:
        df_new = pd.DataFrame([{
            "Pool": c["pool"], "Selection": c["name"], "Model P%": c["pm"]*100.0,
            "Tote P%": c["pt"]*100.0, "Entry Dec": c["dec"], "Entry ROI %": c["roi"]*100.0
        } for c in new_orders])
        display(HTML("<b>Orders placed this snapshot</b>"))
        display(hide_index(df_new.style.format({"Model P%":"{:.2f}%","Tote P%":"{:.2f}%",
                                                "Entry Dec":"{:.2f}","Entry ROI %":"{:+.1f}%"})))

# -----------------------------
# Settlement at the off (t=0)  — apply “half‑edge” late money to OUR bets
# -----------------------------
# Current tote state before the “other syndicates” step
p_win_final, p_exa_final, p_qin_up_final, p_qin_full_final = probs_from_stakes(win_stakes, ex_stakes, qin_stakes_up)

# Derive the unique sets we bet in each pool
picks_win = sorted({b["i"] for b in placed if b["pool"]=="WIN"})
picks_exa = sorted({(b["i"],b["j"]) for b in placed if b["pool"]=="EXA"})
picks_qin = sorted({(b["i"],b["j"]) for b in placed if b["pool"]=="QIN"})

# Force ROI_final = 0.5 * ROI_entry for our picks (never negative)
win_stakes    = force_half_edge_WIN(win_stakes, picks_win, p_model, WIN_PAYOUT)
ex_stakes     = force_half_edge_EXA(ex_stakes, picks_exa, ex_model, EXA_PAYOUT)
qin_stakes_up = force_half_edge_QIN(qin_stakes_up, picks_qin, qin_up_mod, QIN_PAYOUT)

# Recompute final post‑slip tote probabilities
p_win_final, p_exa_final, p_qin_up_final, p_qin_full_final = probs_from_stakes(win_stakes, ex_stakes, qin_stakes_up)

# Draw outcome under PL(model)
I = rng.choice(n, p=p_model)
mask = np.ones(n, dtype=bool); mask[I] = False
p2 = p_model[mask]; p2 = p2/p2.sum()
J = (np.nonzero(mask)[0])[np.searchsorted(np.cumsum(p2), rng.random(), side="left")]

# Per‑bet slippage & PnL
rows=[]; total_pnl=0.0
for b in placed:
    # entry statistics
    if b["pool"]=="WIN":
        pm = p_model[b["i"]]; ptE = b["tote_p_entry"]; decE = b["dec_entry"]
        ptF = p_win_final[b["i"]]; decF = tote_decimal(ptF, WIN_PAYOUT)
        roiE = roi_margin(pm, ptE, WIN_PAYOUT); roiF = roi_margin(pm, ptF, WIN_PAYOUT)
        hit  = (b["i"]==I)
    elif b["pool"]=="EXA":
        pm = ex_model[b["i"],b["j"]]; ptE = b["tote_p_entry"]; decE = b["dec_entry"]
        ptF = p_exa_final[b["i"],b["j"]]; decF = tote_decimal(ptF, EXA_PAYOUT)
        roiE = roi_margin(pm, ptE, EXA_PAYOUT); roiF = roi_margin(pm, ptF, EXA_PAYOUT)
        hit  = (b["i"]==I and b["j"]==J)
    else:
        a,bj = (b["i"], b["j"])
        pm = qin_up_mod[a,bj]; ptE = b["tote_p_entry"]; decE = b["dec_entry"]
        ptF = p_qin_up_final[a,bj]; decF = tote_decimal(ptF, QIN_PAYOUT)
        roiE = roi_margin(pm, ptE, QIN_PAYOUT); roiF = roi_margin(pm, ptF, QIN_PAYOUT)
        hit  = ((I==a and J==bj) or (I==bj and J==a))
    net = STAKE_PER_BET*(decF-1.0) if hit else -STAKE_PER_BET
    rows.append({
        "Time": b["time"], "Pool": b["pool"], "Selection": b["label"],
        "Entry Dec": decE, "Final Dec": decF,
        "Entry ROI %": roiE*100.0, "Final ROI %": roiF*100.0,
        "Slippage % (dec)": (decF/decE - 1.0)*100.0,
        "Net": net
    })
    total_pnl += net

df_settle = pd.DataFrame(rows).sort_values(["Time","Pool"]).reset_index(drop=True)

display(HTML(f"<h3>Outcome: 1st = <b>{horses[I]}</b>, 2nd = <b>{horses[J]}</b></h3>"))
display(HTML("<b>Per‑bet settlement (with controlled half‑edge slippage)</b>"))
display(hide_index(df_settle.style.format({
    "Entry Dec":"{:.2f}", "Final Dec":"{:.2f}",
    "Entry ROI %":"{:+.1f}%", "Final ROI %":"{:+.1f}%",
    "Slippage % (dec)":"{:+.2f}%", "Net":"{:.2f}"
})))

turnover = STAKE_PER_BET*len(placed)
roi_total = total_pnl/turnover if turnover>0 else 0.0
display(HTML("<h4>Portfolio totals</h4>"))
display(pd.DataFrame([{
    "Bets placed": len(placed), "Stake per bet": STAKE_PER_BET, "Turnover": round(turnover,2),
    "Total PnL": round(total_pnl,2), "ROI": round(roi_total,4)
}]))

# --- Small bar chart: Entry ROI% vs Final ROI% (top 15 by Entry ROI)
if len(df_settle):
    df_plot = df_settle.copy()
    df_plot["Entry ROI %"] = df_plot["Entry ROI %"].astype(float)
    df_plot = df_plot.sort_values("Entry ROI %", ascending=False).head(15)
    labels = [f'{r["Pool"]}:{r["Selection"]}' for _, r in df_plot.iterrows()]
    x = np.arange(len(labels))
    width = 0.4
    plt.figure(figsize=(10,4.5))
    plt.bar(x - width/2, df_plot["Entry ROI %"], width)
    plt.bar(x + width/2, df_plot["Final ROI %"], width)
    plt.xticks(x, labels, rotation=45, ha='right')
    plt.title("Entry ROI% vs Final ROI% (top 15 by Entry ROI)")
    plt.ylabel("ROI %")
    plt.tight_layout(); plt.show()


In [ ]:
# === Year‑style Monte Carlo: WIN + EXACTA + QUINELLA ==========================
# Correct late-money impact: our selections' ROI_final = SLIP_FRACTION * ROI_entry (never negative).
# Shows Actual vs No-late PnL curves and reports Avg Entry ROI% vs Avg Final ROI%.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# --------------------------
# Parameters
# --------------------------
N_RACES                 = 3000
WIN_PAYOUT              = 0.825
EXA_PAYOUT              = 0.805
QIN_PAYOUT              = 0.825
PROB_MIN_WIN            = 0.05
PROB_MIN_EXA            = 0.02
PROB_MIN_QIN            = 0.03
MAX_NEW_ORDERS_SNAPSHOT = 10
STAKE_PER_BET           = 1.00
TRADE_WINDOW            = (-60, -2)    # only trade between −8s and −2s inclusive
SNAP_TIMES              = [-60, -45, -30, -20, -10, -8, -6, -4, -2, 0]
RNG_SEED                = 4242

# Fraction of edge we KEEP after late money (1.0 = no impact; 0.5 = keep half; 0.3 = keep 30%)
SLIP_FRACTION           = 0.8

# --------------------------
# Context (reuse horses/model if present)
# --------------------------
if "horses" not in globals():
    horses = [f"H{i}" for i in range(1, 13)]
if "win_probs" not in globals():
    win_probs = np.array([0.22, 0.17, 0.12, 0.09, 0.08, 0.07, 0.06, 0.06, 0.05, 0.04, 0.025, 0.015], dtype=float)
p_model = win_probs/np.sum(win_probs)
n = len(horses)

# --------------------------
# Helpers
# --------------------------
def exacta_from_win(p):
    ex = np.full((n, n), np.nan, dtype=float)
    for i in range(n):
        denom = 1.0 - p[i]
        for j in range(n):
            if i==j: continue
            ex[i,j] = p[i]*(p[j]/denom)
    return ex
def qin_from_exacta(ex):
    qin_up = np.full((n, n), np.nan, dtype=float)
    for i in range(n):
        for j in range(i+1, n):
            qin_up[i,j] = ex[i,j] + ex[j,i]
    return qin_up
def probs_from_stakes(win_stakes, ex_stakes, qin_up):
    p_win = win_stakes/np.sum(win_stakes)
    p_exa = ex_stakes/np.nansum(ex_stakes)
    Tq = np.nansum(qin_up); p_q_up = qin_up/Tq
    p_q_full = np.full((n,n), np.nan, dtype=float)
    for i in range(n):
        for j in range(n):
            if i==j: continue
            a,b = (i,j) if i<j else (j,i)
            p_q_full[i,j]=p_q_up[a,b]
    return p_win, p_exa, p_q_up, p_q_full
def tote_decimal(pt, f): return f / pt
def roi_margin(q, t, f): return f*(q/t) - 1.0

# --- Correct half-edge adjusters (embedded here) ---
def _delta_to_hit_target_prob_scalar(s, T, p_target):
    p_target = float(min(max(p_target, 1e-9), 1-1e-9))
    return max((p_target*T - s) / (1.0 - p_target), 0.0)
def force_fraction_edge_WIN(win_stakes, picks, p_model, payout, frac=SLIP_FRACTION):
    T = float(win_stakes.sum()); p_tote = win_stakes / T
    for i in picks:
        q=float(p_model[i]); tE=float(p_tote[i]); roiE=payout*(q/tE)-1.0
        if roiE<=0: continue
        roiT=frac*roiE; tT=payout*q/(1.0+roiT)
        if tT >= p_tote[i]:
            delta=_delta_to_hit_target_prob_scalar(float(win_stakes[i]), T, tT)
            win_stakes[i]+=delta; T+=delta; p_tote=win_stakes/T
    return win_stakes
def force_fraction_edge_EXA(ex_stakes, picks, ex_model, payout, frac=SLIP_FRACTION):
    T=float(np.nansum(ex_stakes)); p_t=ex_stakes/T
    for (i,j) in picks:
        q=float(ex_model[i,j]); tE=float(p_t[i,j]); roiE=payout*(q/tE)-1.0
        if roiE<=0: continue
        roiT=frac*roiE; tT=payout*q/(1.0+roiT)
        if tT >= p_t[i,j]:
            delta=_delta_to_hit_target_prob_scalar(float(ex_stakes[i,j]), T, tT)
            ex_stakes[i,j]+=delta; T+=delta; p_t=ex_stakes/T
    return ex_stakes
def force_fraction_edge_QIN(qin_up, picks_up, qin_up_model, payout, frac=SLIP_FRACTION):
    T=float(np.nansum(qin_up)); p_t=qin_up/T
    for (a,b) in picks_up:
        q=float(qin_up_model[a,b]); tE=float(p_t[a,b]); roiE=payout*(q/tE)-1.0
        if roiE<=0: continue
        roiT=frac*roiE; tT=payout*q/(1.0+roiT)
        if tT >= p_t[a,b]:
            delta=_delta_to_hit_target_prob_scalar(float(qin_up[a,b]), T, tT)
            qin_up[a,b]+=delta; T+=delta; p_t=qin_up/T
    return qin_up

ex_model_base   = exacta_from_win(p_model)
qin_up_mod_base = qin_from_exacta(ex_model_base)

# --------------------------
# One-race simulator
# --------------------------
def simulate_race(race_idx):
    rng = np.random.default_rng(RNG_SEED + race_idx)

    # Pools
    WIN_POOL_TOTAL = 100_000.0
    EXA_POOL_TOTAL = 80_000.0
    QIN_POOL_TOTAL = 80_000.0
    win_stakes = p_model * WIN_POOL_TOTAL
    ex_stakes  = ex_model_base * EXA_POOL_TOTAL
    qin_up     = qin_up_mod_base * QIN_POOL_TOTAL

    # Mild retail biases
    order = np.argsort(-p_model); A,B,C = order[0], order[1], order[2]
    ex_stakes[A,B]*=1.6; ex_stakes[B,A]*=1.3; ex_stakes[A,C]*=1.2; ex_stakes[C,A]*=1.1
    qin_up[min(A,B),max(A,B)]*=1.4; qin_up[min(A,C),max(A,C)]*=1.2
    ex_stakes *= (EXA_POOL_TOTAL/np.nansum(ex_stakes))
    qin_up    *= (QIN_POOL_TOTAL/np.nansum(qin_up))

    # Realised noisy late flows (independent of our picks)
    ln = lambda base: float(base*np.exp(rng.normal(0.0, 0.30)))
    win_flows = {-45:{A:ln(8000)}, -10:{A:ln(15000)}, -5:{B:ln(5000)}}
    exa_flows = {-30:{(A,B):ln(5000),(A,C):ln(3000),(B,A):ln(2500)}, -20:"exa_fan_out_A", -5:"exa_non_A_pairs"}
    qin_flows = {-30:{(min(A,B),max(A,B)):ln(4000),(min(A,C),max(A,C)):ln(2500)}, -20:"qin_fan_out_A", -5:"qin_non_A_pairs"}

    def apply_exa_kw(k):
        nonlocal ex_stakes
        if k=="exa_fan_out_A":
            addA=ln(6000); add2=ln(2000)
            per=addA/(n-1); per2=add2/(n-1)
            for j in range(n):
                if j!=A: ex_stakes[A,j]+=per
            for i in range(n):
                if i!=A: ex_stakes[i,A]+=per2
        elif k=="exa_non_A_pairs":
            ex_stakes[B,C]+=ln(2500); ex_stakes[C,B]+=ln(1800)
    def apply_qin_kw(k):
        nonlocal qin_up
        if k=="qin_fan_out_A":
            add=ln(5000); per=add/(n-1)
            for x in range(n):
                if x==A: continue
                a,b=(min(A,x),max(A,x)); qin_up[a,b]+=per
        elif k=="qin_non_A_pairs":
            nonA=[x for x in range(n) if x!=A]
            i,j = rng.choice(nonA), rng.choice(nonA)
            while j==i: j=rng.choice(nonA)
            k1,m = rng.choice(nonA), rng.choice(nonA)
            while m==k1: m=rng.choice(nonA)
            a,b=(min(i,j),max(i,j)); c,d=(min(k1,m),max(k1,m))
            qin_up[a,b]+=ln(2200); qin_up[c,d]+=ln(1800)

    used_win=set(); used_exa=set(); used_qin=set()
    placed=[]            # entry info for each bet
    sum_entry_roi=0.0;   # diagnostics: mean entry ROI%
    nb=0

    for t in SNAP_TIMES:
        # realised flows at time t
        if t in win_flows:
            for k,amt in win_flows[t].items(): win_stakes[k]+=amt
        if t in exa_flows:
            f=exa_flows[t]
            if isinstance(f,dict):
                for (i,j),amt in f.items(): ex_stakes[i,j]+=amt
            else: apply_exa_kw(f)
        if t in qin_flows:
            f=qin_flows[t]
            if isinstance(f,dict):
                for (i,j),amt in f.items(): qin_up[i,j]+=amt
            else: apply_qin_kw(f)

        p_win, p_exa, p_q_up, _ = probs_from_stakes(win_stakes, ex_stakes, qin_up)

        # Build positive-ROI candidates (entry view)
        cand=[]
        # WIN
        for i in range(n):
            pm=p_model[i]; pt=p_win[i]
            if pm>=PROB_MIN_WIN and i not in used_win:
                roi=roi_margin(pm, pt, WIN_PAYOUT)
                if roi>0: cand.append({"pool":"WIN","i":i,"j":None,"roi":roi,"pm":pm,
                                       "pt":pt,"dec":tote_decimal(pt,WIN_PAYOUT)})
        # EXA
        for i in range(n):
            for j in range(n):
                if i==j or (i,j) in used_exa: continue
                pm=ex_model_base[i,j]
                if pm<PROB_MIN_EXA: continue
                pt=p_exa[i,j]; roi=roi_margin(pm, pt, EXA_PAYOUT)
                if roi>0: cand.append({"pool":"EXA","i":i,"j":j,"roi":roi,"pm":pm,
                                       "pt":pt,"dec":tote_decimal(pt,EXA_PAYOUT)})
        # QIN (unordered i<j)
        for i in range(n):
            for j in range(i+1,n):
                if (i,j) in used_qin: continue
                pm=qin_up_mod_base[i,j]
                if pm<PROB_MIN_QIN: continue
                pt=p_q_up[i,j]; roi=roi_margin(pm, pt, QIN_PAYOUT)
                if roi>0: cand.append({"pool":"QIN","i":i,"j":j,"roi":roi,"pm":pm,
                                       "pt":pt,"dec":tote_decimal(pt,QIN_PAYOUT)})
        cand.sort(key=lambda x:(x["roi"],x["pm"]), reverse=True)

        # Trade only in late window
        if (TRADE_WINDOW[0] <= t <= TRADE_WINDOW[1]):
            for c in cand[:MAX_NEW_ORDERS_SNAPSHOT]:
                nb += 1
                if c["pool"]=="WIN":   used_win.add(c["i"]);          win_stakes[c["i"]]+=STAKE_PER_BET
                elif c["pool"]=="EXA": used_exa.add((c["i"],c["j"]));  ex_stakes[c["i"],c["j"]]+=STAKE_PER_BET
                else:                  used_qin.add((c["i"],c["j"]));  qin_up[c["i"],c["j"]]+=STAKE_PER_BET
                placed.append({"pool":c["pool"],"i":c["i"],"j":c["j"],
                               "pm":c["pm"],"pt_entry":c["pt"],"dec_entry":c["dec"]})
                sum_entry_roi += c["roi"]

    # Final tote before “half-edge” (for No‑late we use entry decs per bet)
    p_win_f, p_exa_f, p_q_up_f, _ = probs_from_stakes(win_stakes, ex_stakes, qin_up)

    # Apply “half-edge” to our picks to get Actual final
    picks_win = sorted({b["i"] for b in placed if b["pool"]=="WIN"})
    picks_exa = sorted({(b["i"],b["j"]) for b in placed if b["pool"]=="EXA"})
    picks_qin = sorted({(b["i"],b["j"]) for b in placed if b["pool"]=="QIN"})
    win_stakes = force_fraction_edge_WIN(win_stakes, picks_win, p_model, WIN_PAYOUT, frac=SLIP_FRACTION)
    ex_stakes  = force_fraction_edge_EXA(ex_stakes, picks_exa, ex_model_base, EXA_PAYOUT, frac=SLIP_FRACTION)
    qin_up     = force_fraction_edge_QIN(qin_up, picks_qin, qin_up_mod_base, QIN_PAYOUT, frac=SLIP_FRACTION)
    p_win_ff, p_exa_ff, p_q_up_ff, _ = probs_from_stakes(win_stakes, ex_stakes, qin_up)

    # Draw outcome under PL(model)
    rng_out = np.random.default_rng(RNG_SEED*7 + race_idx)
    I = rng_out.choice(n, p=p_model)
    mask=np.ones(n,dtype=bool); mask[I]=False
    p2=p_model[mask]; p2=p2/p2.sum()
    J=(np.nonzero(mask)[0])[np.searchsorted(np.cumsum(p2), rng_out.random(), side="left")]

    # Settle: Actual vs No‑late and measure Avg final ROI
    pnl_actual=0.0; pnl_nolate=0.0; sum_final_roi=0.0
    for b in placed:
        if b["pool"]=="WIN":
            ptF = p_win_ff[b["i"]]; decF = tote_decimal(ptF, WIN_PAYOUT)
            hit = (b["i"]==I)
            sum_final_roi += roi_margin(b["pm"], ptF, WIN_PAYOUT)
        elif b["pool"]=="EXA":
            ptF = p_exa_ff[b["i"],b["j"]]; decF = tote_decimal(ptF, EXA_PAYOUT)
            hit = (b["i"]==I and b["j"]==J)
            sum_final_roi += roi_margin(b["pm"], ptF, EXA_PAYOUT)
        else:
            a,bj=(b["i"],b["j"])
            ptF = p_q_up_ff[a,bj]; decF = tote_decimal(ptF, QIN_PAYOUT)
            hit = ((I==a and J==bj) or (I==bj and J==a))
            sum_final_roi += roi_margin(b["pm"], ptF, QIN_PAYOUT)

        pnl_actual += STAKE_PER_BET*(decF-1.0) if hit else -STAKE_PER_BET
        pnl_nolate += STAKE_PER_BET*(b["dec_entry"]-1.0) if hit else -STAKE_PER_BET

    avg_entry_roi = (sum_entry_roi / nb) if nb>0 else 0.0
    avg_final_roi = (sum_final_roi / nb) if nb>0 else 0.0
    return pnl_actual, pnl_nolate, nb, avg_entry_roi, avg_final_roi

# --------------------------
# Run Monte Carlo
# --------------------------
display(HTML(f"<h3>Year‑style Monte Carlo — late money keeps {int(SLIP_FRACTION*100)}% of our entry edge</h3>"))
display(HTML(f"<em>N_RACES={N_RACES:,}; trade window {TRADE_WINDOW[0]}s to {TRADE_WINDOW[1]}s; "
             f"filters: WIN ≥{PROB_MIN_WIN:.0%}, EXA ≥{PROB_MIN_EXA:.0%}, QIN ≥{PROB_MIN_QIN:.0%}</em>"))

per_race_actual=[]; per_race_nolate=[]; total_bets=0
entry_roi_acc=0.0; final_roi_acc=0.0

for r in range(N_RACES):
    a, nl, nb, e_roi, f_roi = simulate_race(r)
    per_race_actual.append(a); per_race_nolate.append(nl)
    total_bets += nb; entry_roi_acc += e_roi*nb; final_roi_acc += f_roi*nb

per_race_actual = np.array(per_race_actual, dtype=float)
per_race_nolate = np.array(per_race_nolate, dtype=float)
cum_actual = np.cumsum(per_race_actual)
cum_nolate = np.cumsum(per_race_nolate)
lost_to_late = cum_nolate[-1] - cum_actual[-1]

turnover = STAKE_PER_BET*total_bets
roi_actual  = (cum_actual[-1]/turnover) if turnover>0 else 0.0
roi_nolate  = (cum_nolate[-1]/turnover) if turnover>0 else 0.0
avg_entry_roi = (entry_roi_acc/max(total_bets,1))*100.0
avg_final_roi = (final_roi_acc/max(total_bets,1))*100.0

summary = pd.DataFrame([
    ["Races simulated", f"{N_RACES:,}"],
    ["Total bets placed", f"{total_bets:,}"],
    ["Stake per bet", f"{STAKE_PER_BET:.2f}"],
    ["Total turnover", f"{turnover:,.2f}"],
    ["Total PnL (Actual)", f"{cum_actual[-1]:,.2f}"],
    ["Total ROI (Actual)", f"{roi_actual:.4f}"],
    ["Total PnL (No‑late)", f"{cum_nolate[-1]:,.2f}"],
    ["Total ROI (No‑late)", f"{roi_nolate:.4f}"],
    ["Edge lost to late money", f"{lost_to_late:,.2f}"],
    ["Avg Entry ROI per bet", f"{avg_entry_roi:+.2f}%"],
    ["Avg Final ROI per bet", f"{avg_final_roi:+.2f}%"],
], columns=["Metric","Value"])
display(summary)

# Plots
plt.figure(figsize=(10,4.8))
plt.plot(cum_actual, label="Actual (after late money)")
plt.plot(cum_nolate, label="No‑late (freeze at entry)")
plt.title(f"Cumulative PnL — Actual vs No‑late  •  late money keeps {int(SLIP_FRACTION*100)}% of edge")
plt.xlabel("Race #"); plt.ylabel("Cumulative PnL"); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(9,4.2))
plt.hist(per_race_actual, bins=80)
plt.title("Per‑race PnL distribution (Actual)")
plt.xlabel("PnL per race"); plt.ylabel("Frequency"); plt.tight_layout(); plt.show()
